In [ ]:
# Using the vite plugin:
# https://github.com/justineo/vite-plugin-import-graph
# to create call graphs from vite projects

In [4]:
project_dir = "./data/cula"

In [5]:
# load jsons from this folder
import json
import os
import re

# find all jsons in this folder
jsons = [f for f in os.listdir(project_dir) if re.match(r'.*\.json', f) if not f.startswith('_')]
jsons

['service-third-party-integration-client-import-graph.json',
 'service-tracking-client-import-graph.json',
 'service-provider-client-import-graph.json',
 'service-registry-client-import-graph.json']

In [7]:
# read the jsons and store them in a dictionary
data = {}
for sub_project in jsons:
    with open(os.path.join(project_dir, sub_project), 'r') as f:
        data[sub_project] = json.load(f)

# print the keys of the dictionary
data.keys()

dict_keys(['service-third-party-integration-client-import-graph.json', 'service-tracking-client-import-graph.json', 'service-provider-client-import-graph.json', 'service-registry-client-import-graph.json'])

In [16]:
path_prefix = "/Users/hendrik/Documents/cula/"

# join all the jsons into one. Every value is an array. Join arrays on same key
joined = {}
for sub_project in jsons:
    for k in data[sub_project]:
        # skip virtual and installed packages
        if not k.startswith(path_prefix):
            continue
        # remove the first part of the key "/Users/hendrik/Documents/cula/"
        rel_key = k.replace(path_prefix, '')
        
        if rel_key not in joined:
            joined[rel_key] = []
            
        import_list = data[sub_project][k]
        # remove the first part of the values "/Users/hendrik/Documents/cula/"
        import_list = [target.replace(path_prefix, '') 
                       for target in import_list 
                       if target.startswith(path_prefix)]
        
        joined[rel_key].extend(import_list)

In [25]:
# save joined json
with open(os.path.join(project_dir, '_joined.json'), 'w') as f:
    json.dump(joined, f, indent=4)
    
# current format for all files in which fileX imports fileY
# {
#     'fileX': [
#         'fileY'
#     ]
# }

In [26]:
# now generate it in the following format for each fileX
# {
#     "file": rel_path,
#     "calls": list([rel_path, ...]),
#     "called_by": list([rel_path, ...])
# }

# generate the new format fileA imports fileB
graph = []
for importer_file in joined:
    calls = joined[importer_file]
    called_by = []
    for imported_file in joined:
        if importer_file in joined[imported_file]:
            called_by.append(imported_file)
    graph.append({
        "file": importer_file,
        "calls": calls,
        "called_by": called_by
    })

In [27]:
# save as _graph.json
with open(os.path.join(project_dir, '_graph.json'), 'w') as f:
    json.dump(graph, f, indent=4)

In [28]:
from setup_repository import store_call_analysis_results, load_call_analysis_results

store_call_analysis_results('cula', graph)

In [29]:
load_call_analysis_results('cula')

[{'file': 'cula-platform/packages/service-third-party-integration-client/index.html',
  'calls': ['cula-platform/packages/service-third-party-integration-client/src/app/main.ts',
   'cula-platform/packages/service-third-party-integration-client/src/app/main.ts'],
  'called_by': []},
 {'file': 'cula-platform/packages/service-third-party-integration-client/src/app/main.ts',
  'calls': ['cula-platform/packages/service-third-party-integration-client/src/app/App.vue',
   'cula-platform/packages/service-third-party-integration-client/src/app/index.css',
   'cula-platform/packages/service-third-party-integration-client/src/app/App.vue',
   'cula-platform/packages/service-third-party-integration-client/src/app/index.css',
   'cula-platform/packages/service-third-party-integration-client/src/app/router/router.ts',
   'cula-platform/packages/service-third-party-integration-client/src/app/router/router.ts'],
  'called_by': ['cula-platform/packages/service-third-party-integration-client/index.html